# Note:
    Store features on the cloud

# Import libs

In [6]:
import os, sys
import pickle

import pandas as pd
import numpy as np
from joblib import dump, load
# import texthero
from sklearn.feature_extraction.text import TfidfVectorizer
from sqlalchemy import create_engine

sys.path.append("..")
from others.database import SessionLocal, engine
from sqlalchemy import desc, func, asc
from others.schema import FeedGet, PostGet, UserGet
from others.table_post import Post
from others.table_user import User
from others.table_feed import Feed

import matplotlib.pyplot as plt

# Import data

In [7]:
df_users = pd.read_parquet('df_users_embedings.parquet')
df_posts = pd.read_parquet('df_posts_new_features_and_post.parquet')

In [26]:
df_users.rename(columns={"id":"user_id"}, inplace=True)

# Save data on the cloud

In [27]:
def load_data_users(df, engine) -> int:
    chunksize = 200000
    return df.to_sql(name='aleksandr_samofalov_features_lesson_22_users_october_2023',
                  con=engine, chunksize=chunksize, if_exists='replace')

def load_data_post(df, engine) -> int:
    chunksize = 200000
    return df.to_sql(name='aleksandr_samofalov_features_lesson_22_post_october_2023',
                  con=engine, chunksize=chunksize, if_exists='replace')

In [28]:
load_data_users(df_users, engine)

205

In [12]:
load_data_post(df_posts, engine)

23

# Check saved data

In [15]:
def batch_load_sql(query: str, engine: object) -> pd.DataFrame:
    CHUNKSIZE = 200000
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for ind, chunk_dataframe in enumerate(pd.read_sql(query, conn, chunksize=CHUNKSIZE)):
        print(chunk_dataframe.shape)
        print(ind)
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [16]:
def load_features_post() -> pd.DataFrame:
    engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
    )
    df_post_data = batch_load_sql("""SELECT * FROM aleksandr_samofalov_features_lesson_22_post_october_2023""", engine)
    df_post_data = df_post_data.drop(['index'], axis=1)
    return df_post_data


def load_features() -> pd.DataFrame:
    engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
    )
    df_user_data = batch_load_sql("""SELECT * FROM aleksandr_samofalov_features_lesson_22_users_october_2023""", engine)
    df_user_data = df_user_data.drop(['index'], axis=1)
    return df_user_data

In [17]:
load_features_post()

(7023, 26)
0


,post_id,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,kmean_label,DistanceToCluster_0,DistanceToCluster_1,...,DistanceToCluster_7,DistanceToCluster_8,DistanceToCluster_9,DistanceToCluster_10,DistanceToCluster_11,DistanceToCluster_12,DistanceToCluster_13,DistanceToCluster_14,text,topic
0,1,False,False,False,False,False,False,2,3.729024,2.998742,...,3.417403,3.373112,2.350711,1.900091,2.218462,2.839048,3.461114,3.412472,UK economy facing major risks\n\nThe UK manufa...,business
1,2,False,False,False,False,False,False,2,3.531465,2.843195,...,3.323233,3.324048,2.316050,2.180061,2.233397,2.555996,3.138126,3.218686,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,False,False,False,False,False,False,2,3.538897,3.058267,...,3.348733,3.265760,2.390932,1.806858,3.036396,2.888302,3.141400,3.283395,Asian quake hits European shares\n\nShares in ...,business
3,4,False,False,False,False,False,False,10,3.031447,3.260913,...,3.731672,3.517551,2.811656,2.429931,3.393587,3.379051,3.792460,3.695954,India power shares jump on debut\n\nShares in ...,business
4,5,False,False,False,False,False,False,10,3.280477,2.642357,...,2.803656,3.028448,2.009257,1.457895,2.923765,2.129916,2.764996,2.838905,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,False,False,True,False,False,False,7,2.336807,2.816310,...,1.283884,3.135332,2.738250,3.003807,3.355360,2.330962,1.821759,2.005075,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,False,False,True,False,False,False,7,2.197449,2.500667,...,0.940390,2.930152,2.448724,2.967479,3.189711,2.229802,1.843959,1.767640,I give this movie 2 stars purely because of it...,movie
7020,7317,False,False,True,False,False,False,7,2.626637,2.534971,...,1.491582,2.834718,2.810127,3.188147,3.403540,2.444165,1.992768,2.170022,I cant believe this film was allowed to be mad...,movie
7021,7318,False,False,True,False,False,False,6,1.975253,3.106200,...,1.489237,3.432551,2.991727,3.197914,3.443767,2.307946,1.514832,1.855331,The version I saw of this film was the Blockbu...,movie


In [29]:
load_features()

(163205, 22)
0


,user_id,gender,age,Belarus,Cyprus,Estonia,Finland,Kazakhstan,Latvia,Russia,...,Turkey,Ukraine,exp_1,exp_2,exp_3,exp_4,Android,iOS,ads,organic
0,200,1,34,False,False,False,False,False,False,True,...,False,False,False,False,True,False,True,False,True,False
1,201,0,37,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,True,False
2,202,1,17,False,False,False,False,False,False,True,...,False,False,False,False,False,True,True,False,True,False
3,203,0,18,False,False,False,False,False,False,True,...,False,False,True,False,False,False,False,True,True,False
4,204,0,36,False,False,False,False,False,False,True,...,False,False,False,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,False,False,False,False,False,False,True,...,False,False,False,False,False,True,True,False,False,True
163201,168549,0,18,False,False,False,False,False,False,True,...,False,False,False,True,False,False,True,False,False,True
163202,168550,1,41,False,False,False,False,False,False,True,...,False,False,False,False,False,True,True,False,False,True
163203,168551,0,38,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,True,False,True
